<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/nlpInternship/blob/main/POSTaggerMbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.47.0 datasets seqeval scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=fb20983a4869598a486130f43d132abbce80ed2a005f8935125be153fe3fff26
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERRO

In [2]:
import ast  # safely parse string to Python object
import numpy as np
import transformers
from seqeval.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback  # <-- Add this import
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset, DatasetDict
import time
start = time.time()

2025-05-16 06:58:52.739050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747378732.939205      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747378732.997762      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def extract_tokens_and_tags(example):
    ann = ast.literal_eval(example['Annotated by: Annotator 1 '])  # safely convert string to list of dicts
    tokens = [entry["word"] for entry in ann]
    tags = [entry["entity"] for entry in ann]
    return {"tokens": tokens, "tags": tags}
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2tag[label] for label in sent if label != -100]
        for sent in labels
    ]
    true_predictions = [
        [id2tag[pred] for (pred, label) in zip(sent_preds, sent_labels) if label != -100]
        for sent_preds, sent_labels in zip(predictions, labels)
    ]
    print(classification_report(true_labels,true_predictions,digits = 6))
    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions)
    }
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128

    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs



In [4]:
# Load POS task subset

from datasets import load_dataset
dataset = load_dataset("LingoIITGN/COMI-LINGUA", "POS")

# Convert to tokens and tags
processed_data = dataset.map(extract_tokens_and_tags)

# Keep only necessary columns
processed_data = processed_data.remove_columns([
    "Sentences", "Predicted Tags",
    "Annotated by: Annotator 1 ",
    "Annotated by: Annotator 2",
    "Annotated by: Annotator 3"
])


print(len(processed_data['train']))

# Assuming processed_data is your Dataset/DatasetDict
data = processed_data['train']

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

POS_train.csv:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

POS_test.csv:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15684 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4999 [00:00<?, ? examples/s]

Map:   0%|          | 0/15684 [00:00<?, ? examples/s]

Map:   0%|          | 0/4999 [00:00<?, ? examples/s]

15684


In [5]:
# type(dataset['train'][15683]) # dataset['train'][i] the annotaion by all the annotators for the the sentence

In [6]:
# First split: 80% train, 20% temp
train_test_split = data.train_test_split(
    test_size=0.2,
    seed=42
)

# Second split: 10% val, 10% test
val_test_split = train_test_split['test'].train_test_split(
    test_size=0.5,
    seed=42
)

# Combine
processed_data = DatasetDict({
    'train': train_test_split['train'],
    'validation': val_test_split['train'],
    'test': val_test_split['test']
})

In [7]:
print(len(processed_data['train']))
print(len(processed_data['validation']))
print(len(processed_data['test']))

12547
1568
1569


In [8]:
processed_data['train'][0]

{'tokens': ['नई',
  'दिल्ली',
  ':',
  'देश',
  'की',
  'प्रमुख',
  'दो',
  '-',
  'पहिया',
  'वाहन',
  'निर्माता',
  'कंपनी',
  'टीवीएस',
  'मोटर्स',
  '(',
  'TVS',
  'Motors',
  ')',
  'आज',
  'यानी',
  'सोमवार',
  'को',
  'Apache',
  'RR',
  '310',
  '2021',
  'को',
  'लॉन्च',
  'करने',
  'जा',
  'रही',
  'है',
  '।'],
 'tags': ['PROPN',
  'PROPN',
  'X',
  'NOUN',
  'ADP',
  'ADJ',
  'ADJ',
  'X',
  'ADJ',
  'NOUN',
  'NOUN',
  'NOUN',
  'PROPN',
  'PROPN',
  'X',
  'PROPN',
  'PROPN',
  'X',
  'ADV',
  'CONJ',
  'PROPN',
  'ADP',
  'NOUN',
  'NOUN',
  'NUM',
  'NUM',
  'ADP',
  'VERB',
  'VERB',
  'VERB',
  'VERB',
  'VERB',
  'X']}

In [9]:
# Get all unique tags
unique_tags = list(set(tag for row in data for tag in row['tags']))
tag2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
id2tag = {i: tag for tag, i in tag2id.items()}
NUM_LABELS = len(tag2id)


In [10]:
print(len(id2tag))

14


In [11]:
# Tokenize

model_checkpoint = "google/muril-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=NUM_LABELS,
    id2label=id2tag,
    label2id=tag2id
)

tokenized_train = processed_data['train'].map(tokenize_and_align_labels, batched=True)
tokenized_val = processed_data['validation'].map(tokenize_and_align_labels, batched=True)
tokenized_test = processed_data['test'].map(tokenize_and_align_labels, batched=True)

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/12547 [00:00<?, ? examples/s]

Map:   0%|          | 0/1568 [00:00<?, ? examples/s]

Map:   0%|          | 0/1569 [00:00<?, ? examples/s]

In [12]:
tokenized_train

Dataset({
    features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 12547
})

In [13]:
# Create training arguements
training_args = TrainingArguments(
    output_dir="./muril-pos-results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    load_best_model_at_end=True,  # <-- Important for early stopping
    metric_for_best_model="eval_loss",  # <-- Metric to monitor
    greater_is_better=False  # <-- For loss (lower is better)
)

# Create Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,  # Stop after 3 epochs without improvement
            early_stopping_threshold=0.01  # Minimum improvement to qualify as better
        )
    ],
    compute_metrics=compute_metrics

)


In [14]:
t = trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.208000,2.119330,0.910696,0.858562
2,1.761600,1.677886,0.924890,0.879143
3,1.351100,1.294264,0.929979,0.887258
4,1.034900,1.004369,0.932064,0.889765
5,0.806400,0.802089,0.932463,0.891015
6,0.641400,0.659456,0.935007,0.894377
7,0.516800,0.559938,0.934977,0.894728
8,0.450800,0.492163,0.934241,0.893851
9,0.374400,0.437464,0.936479,0.896978
10,0.317000,0.399417,0.936019,0.896679


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: Us

              precision    recall  f1-score   support

         ART   0.876344  0.652000  0.747706       250
     ART_NEG   0.956311  0.989950  0.972840       199
          DJ   0.770092  0.720715  0.744586      1622
          DP   0.943791  0.970261  0.956843      4136
          DV   0.854060  0.763787  0.806405      1088
         ERB   0.866224  0.851944  0.859025      3215
          ET   0.867742  0.901173  0.884141       597
         ONJ   0.937322  0.918994  0.928068       716
         OUN   0.820644  0.842455  0.831406      5719
         RON   0.916039  0.922993  0.919503       922
      RON_WH   0.925439  0.950450  0.937778       222
        ROPN   0.879630  0.886856  0.883228      2678
          UM   0.944043  0.912740  0.928128       573
           _   0.945111  0.938991  0.942041      2934

   micro avg   0.880118  0.878171  0.879143     24871
   macro avg   0.893057  0.873094  0.881550     24871
weighted avg   0.879664  0.878171  0.878495     24871



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/

...

              precision    recall  f1-score   support

         ART   0.905579  0.844000  0.873706       250
     ART_NEG   0.965854  0.994975  0.980198       199
          DJ   0.771254  0.777435  0.774332      1622
          DP   0.961603  0.968810  0.965193      4136
          DV   0.835974  0.824449  0.830171      1088
         ERB   0.877898  0.883359  0.880620      3215
          ET   0.868463  0.917923  0.892508       597
         ONJ   0.940701  0.974860  0.957476       716
         OUN   0.856460  0.846127  0.851262      5719
         RON   0.920213  0.938178  0.929108       922
      RON_WH   0.968468  0.968468  0.968468       222
        ROPN   0.896858  0.905900  0.901356      2678
          UM   0.945804  0.944154  0.944978       573
           _   0.960274  0.955692  0.957977      2934

   micro avg   0.896510  0.898637  0.897572     24871
   macro avg   0.905386  0.910309  0.907668     24871
weighted avg   0.896364  0.898637  0.897443     24871



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/

              precision    recall  f1-score   support

         ART   0.894515  0.848000  0.870637       250
     ART_NEG   0.965854  0.994975  0.980198       199
          DJ   0.771077  0.772503  0.771789      1622
          DP   0.960874  0.967843  0.964346      4136
          DV   0.827206  0.827206  0.827206      1088
         ERB   0.886115  0.883359  0.884735      3215
          ET   0.861852  0.919598  0.889789       597
         ONJ   0.930667  0.974860  0.952251       716
         OUN   0.855559  0.850323  0.852933      5719
         RON   0.927724  0.932755  0.930233       922
      RON_WH   0.968468  0.968468  0.968468       222
        ROPN   0.891456  0.907767  0.899537      2678
          UM   0.946996  0.935428  0.941176       573
           _   0.960083  0.950920  0.955479      2934

   micro avg   0.895891  0.898557  0.897222     24871
   macro avg   0.903460  0.909572  0.906341     24871
weighted avg   0.895846  0.898557  0.897140     24871



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/

              precision    recall  f1-score   support

         ART   0.894958  0.852000  0.872951       250
     ART_NEG   0.961165  0.994975  0.977778       199
          DJ   0.781659  0.772503  0.777054      1622
          DP   0.959942  0.967602  0.963757      4136
          DV   0.832562  0.827206  0.829876      1088
         ERB   0.882791  0.885537  0.884161      3215
          ET   0.857585  0.927973  0.891392       597
         ONJ   0.943243  0.974860  0.958791       716
         OUN   0.859292  0.848925  0.854077      5719
         RON   0.923901  0.934924  0.929380       922
      RON_WH   0.964286  0.972973  0.968610       222
        ROPN   0.891447  0.910754  0.900997      2678
          UM   0.950530  0.938918  0.944688       573
           _   0.959959  0.956033  0.957992      2934

   micro avg   0.897390  0.899843  0.898615     24871
   macro avg   0.904523  0.911799  0.907965     24871
weighted avg   0.897157  0.899843  0.898426     24871



In [15]:
metrics = trainer.evaluate(eval_dataset=tokenized_test)
print(metrics)


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: Us

              precision    recall  f1-score   support

         ART   0.806167  0.820628  0.813333       223
     ART_NEG   0.980296  0.970732  0.975490       205
          DJ   0.788091  0.789474  0.788782      1710
          DP   0.969578  0.971485  0.970530      4068
          DV   0.827195  0.857982  0.842308      1021
         ERB   0.900796  0.899969  0.900383      3269
          ET   0.877483  0.896785  0.887029       591
         ONJ   0.942857  0.959049  0.950884       757
         OUN   0.863813  0.848123  0.855896      5781
         RON   0.919505  0.936909  0.928125       951
      RON_WH   0.955882  0.951220  0.953545       205
        ROPN   0.885672  0.916572  0.900857      2637
          UM   0.939683  0.959481  0.949479       617
           _   0.961290  0.950319  0.955773      2979

   micro avg   0.900921  0.902974  0.901947     25014
   macro avg   0.901308  0.909195  0.905172     25014
weighted avg   0.900988  0.902974  0.901910     25014

{'eval_loss': 0.295327454

In [16]:
metrics

{'eval_loss': 0.29532745480537415,
 'eval_accuracy': 0.9389873725109277,
 'eval_f1': 0.9019466906259359,
 'eval_runtime': 11.8244,
 'eval_samples_per_second': 132.692,
 'eval_steps_per_second': 4.229,
 'epoch': 17.0}

In [17]:
end = time.time()
print(f"Time taken : {(end-start)/60} mins")

Time taken : 77.46967084407807 mins
